In [177]:
#!pip install plotly
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json
#!pip install nbformat
import nbformat

## This is a notebook used just to visualise the solution and create the chart in the output folder

In [178]:
# graph schedule with demand

# read output
df = pd.read_csv("shift.csv")

# Convert to datetime format
df['start_date_time'] = pd.to_datetime(df['start_date_time'])
df['end_date_time'] = pd.to_datetime(df['end_date_time'])
df['staff_id'] = df['staff_id'].astype(str)

# Calculate the duration of each shift in hours
df['shift_duration'] = (df['end_date_time'] - df['start_date_time']).dt.total_seconds() / 3600

fig = px.timeline(df.sort_values(by=['staff_id'], ascending=False), 
                  x_start='start_date_time', 
                  x_end='end_date_time', 
                  y='staff_id', 
                  title='Staff Work Shifts',
                  labels={'staff_id': 'Staff', 'start_date_time': 'Shift Start', 'end_date_time': 'Shift End'},
                  color='staff_id',
                  text='shift_duration')

fig = go.Figure(fig)

# read demand
demand_df = pd.read_csv("../data/demand.csv")
demand_df['date_time'] = pd.to_datetime(demand_df['date_time'])

fig.add_trace(go.Scatter(
    x=demand_df['date_time'],
    y=demand_df['demand'],
    mode="lines",
    name="Demand",
    line=dict(color="gray", width=1),
    yaxis="y2"  # Secondary y-axis
))

# add metrics in title
with open("metrics.json") as f:
    metrics = json.load(f)


# Update layout to add and configure the secondary y-axis
fig.update_layout(
    yaxis=dict(
        title="Staff",
        anchor="x",
        domain=[0.0, 1.0]  # Ensure it spans the full height
    ),
    yaxis2=dict(
        title="Demand",
        overlaying="y",  # Share the same x-axis
        side="right"     # Display the secondary y-axis on the right
    ),
    xaxis_title="Time",
    title=f"Staff Schedule with Demand on Y-Axis WDC {metrics['WDC']}, WOR {metrics['WOR']}",
    showlegend=False
)

fig.show()

In [179]:
# graph demand actual vs demand

# Expand `start_date` and `end_date` into hourly intervals
expanded_rows = []
for _, row in df.iterrows():
    expanded_hours = pd.date_range(row['start_date_time'], row['end_date_time'], freq='h')
    expanded_rows.extend([{'date_time': dt, 'count': 1, 'staff_id': row['staff_id']} for dt in expanded_hours if dt<row['end_date_time']])

df_expanded = pd.DataFrame(expanded_rows).groupby(['date_time', 'staff_id'], as_index=False).count()
df_pivot = df_expanded.pivot(index='date_time', columns='staff_id', values='count').fillna(0)

# Create the figure
fig = go.Figure()

# Add stacked bars for each staff_id
for staff_id in sorted(df_pivot.columns, reverse=True):
    fig.add_trace(go.Bar(
        x=df_pivot.index,
        y=df_pivot[staff_id],
        name=f'Staff {staff_id}',
    ))

# Add demand as a line
fig.add_trace(go.Scatter(
    x=demand_df['date_time'],
    y=demand_df['demand'],
    mode='lines',
    name='Demand',
    line=dict(color='gray', width=2)
))

# Add vertical lines for the start of each day
start_of_days = pd.to_datetime(df_pivot.index.normalize()).drop_duplicates()
for day_start in start_of_days:
    fig.add_vline(
        x=day_start,
        line=dict(color='black', dash='dot')
    )

# Update layout
fig.update_layout(
    title='Staff Allocation vs Demand',
    xaxis_title='Date Time',
    yaxis_title='Count',
    barmode='stack',  # Stacked bar chart
    template='plotly_white'
)

# Show the figure
fig.show()
